# このノートブックについて
Python 用最適化ライブラリ optuna の使用例を示す．  
optuna の他に sklearn.model_selection.GridSearchCV などもある（詳細は課題図書参照）

# 目次

1. データの準備
2. optuna を用いないモデリング
3. optuna を用いたモデリング

# 利用 Library, ノートブック設定等

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

# 1. データの準備

米国ボストン市郊外における地域別の住宅価格のデータセット
- [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_diabetes.html#sklearn.datasets.load_diabetes)
- [japanese](https://pythondatascience.plavox.info/scikit-learn/scikit-learn%E3%81%AB%E4%BB%98%E5%B1%9E%E3%81%97%E3%81%A6%E3%81%84%E3%82%8B%E3%83%87%E3%83%BC%E3%82%BF%E3%82%BB%E3%83%83%E3%83%88)

In [ ]:
dataset = load_boston()
X = pd.DataFrame(dataset.data, columns=dataset.feature_names)
y = dataset.target
print(X.shape)
display(X.head())
print(y.shape)
display(y)

# 学習・予測用に分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2,
                                                    random_state=123)

# 2. optuna を用いないモデル作成

In [ ]:
# パラメータを指定せず学習・予測
model = DecisionTreeRegressor(random_state=123)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# 評価（RMSE）
np.sqrt(mean_squared_error(y_test, y_pred))
# 約 6.82 千ドルの誤差で住宅価格を予測

# 3. optuna を用いたモデル作成

In [ ]:
import optuna

def objective(trial):
    # パラメータ定義部分
    # 変数名は任意だが慣習として設定先のモデルパラメータと同名にすることが多い
    # suggest_int の第一引数で，モデルパラメータ名，第二・三引数で探索範囲を指定する
    # suggest_* のメソッドは他にもある．
    max_depth         = trial.suggest_int('max_depth', 2, 256)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 256)
    max_leaf_nodes    = trial.suggest_int('max_leaf_nodes', 2, 256)
    criterion         = trial.suggest_categorical('criterion', ['mse', 'friedman_mse'])
    # 学習・予測
    model = DecisionTreeRegressor(
        random_state=123,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        max_leaf_nodes=max_leaf_nodes,
        criterion=criterion
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # 簡単のため RMSE を算出
    return np.sqrt(mean_squared_error(y_test, y_pred))

study = optuna.create_study()
study.optimize(objective, n_trials=10)

print(study.best_params)
print(study.best_value)
# 約 5.9 千ドルの誤差で住宅価格を予測

# 参考
- [時系列データの順番を保持したまま、訓練データとテストデータに分ける](https://qiita.com/Umaremin/items/7055981d7fe58569cfde)
- [予測精度評価指標をPythonで実装する](https://qiita.com/23tk/items/2dbb930242eddcbb75fb)
- [決定木のハイパーパラメータをOptunaで最適化する](https://qiita.com/lystahi/items/37452b637efb13f9a796)